# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('churn').getOrCreate()

In [4]:
df = spark.read.csv('customer_churn.csv', header=True, inferSchema=True)

In [6]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [8]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [9]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [10]:
from pyspark.ml.feature import VectorAssembler

In [11]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'], outputCol='features')

In [15]:
output = assembler.transform(df)

In [16]:
final_df = output.select(['features', 'Churn'])

In [12]:
from pyspark.ml.classification import LogisticRegression

In [18]:
model = LogisticRegression(featuresCol='features', labelCol='Churn')

In [17]:
train_data, test_data = final_df.randomSplit([0.7, 0.3])

In [19]:
fitted = model.fit(train_data)

In [34]:
summary = fitted.summary

In [36]:
summary.areaUnderROC

0.9103786551214662

In [20]:
result = fitted.evaluate(test_data)

In [28]:
result.areaUnderROC

0.8976400180668477

In [37]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [39]:
result.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[4.90836725931104...|[0.99266959610265...|       0.0|
|[26.0,8787.39,1.0...|    1|[0.93863705788237...|[0.71882426726535...|       0.0|
|[26.0,8939.61,0.0...|    0|[6.71305789231348...|[0.99878653102314...|       0.0|
|[28.0,8670.98,0.0...|    0|[8.06775111747489...|[0.99968661075814...|       0.0|
|[28.0,11245.38,0....|    0|[4.04600787850517...|[0.98280864639875...|       0.0|
|[29.0,5900.78,1.0...|    0|[4.51222078039795...|[0.98914506066603...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.99201662569369...|[0.95221215874485...|       0.0|
|[29.0,9378.24,0.0...|    0|[5.03133263006632...|[0.99351226302923...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.72793834038699...|[0.99123285565781...|       0.0|
|[29.0,13240.01,

In [40]:
model_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

In [41]:
model_eval.evaluate(result.predictions)

0.8275745257452575